In [1]:
import streamlit as st
# from streamlit_extras.add_vertical_space import add_vertical_space
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [2]:

from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [10]:
# Load documents from a directory
loader = DirectoryLoader("./input-data", glob="**/*.txt")

print("dir loaded loader")

documents = loader.load()

print(len(documents))

# # Create embeddingsclear
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)

dir loaded loader
1


In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," ",""],
    chunk_size=1500,
    chunk_overlap=300,
    add_start_index=True,
)

# # Split documents into chunks
texts = text_splitter.split_documents(documents)

In [12]:
# # Create vector store
vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding= embeddings,
    persist_directory="./db-sales-enquiry")

print("vectorstore created")

OllamaEmbeddings: 100%|██████████| 10/10 [00:31<00:00,  3.10s/it]


vectorstore created


In [14]:
# # Create embeddingsclear
embeddings = OllamaEmbeddings(model="nomic-embed-text", show_progress=False)

db = Chroma(persist_directory="./db-sales-enquiry",
            embedding_function=embeddings)

# # Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

# # Create Ollama language model - Gemma 2
local_llm = 'phi3:latest'

llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=512,  
                 temperature=0)

C:\Users\waghm\AppData\Local\Temp\ipykernel_25756\3627986529.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="./db-sales-enquiry",


In [15]:
# Create prompt template
template = """<bos><start_of_turn>user
You are a sales inquiry assistant specializing in rebates, market share, promotions, and payouts. Answer the question based only on the following context and provide a meaningful response. 
Write in full sentences with correct spelling and punctuation. Use bullet points or numbered lists when appropriate.
If the context doesn't contain the answer, politely state that you don't have the specific information and suggest contacting the sales department for more details.

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model
ANSWER:"""

In [16]:
prompt = ChatPromptTemplate.from_template(template)

# Create the RAG chain using LCEL with prompt printing and streaming output
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [17]:
response = rag_chain.invoke('RfP location is grouped incorrectly')
response.content

"The issue with the incorrect grouping of an Rfp (Rebate for Fixed Price) sales location seems to be related to account enrollment in Alliance, as indicated by your inquiry. To resolve this matter effectively and ensure accurate rebates through Alliance, please take these steps:\n\n1. Ensure that you have signed up on the Acuvue Alliance portal or reach out directly to SA (Sales Agent) for assistance with manual enrollment if necessary. You can contact viseyesite@its.jnj.com as mentioned in your inquiry, but ensure this is done through proper channels and not just by sending an email without prior verification of the process or protocols required to manually join Alliance.\n\n2. Once you have confirmed that your account has been enrolled into Alliance correctly with a common indirect customer reference account number alignment under myAPP/Alliance, please verify if this resolves any discrepan0n in RfP location grouping and rebate calculations within the system. If not resolved or furth

In [10]:

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": prompt}
)

In [7]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
import pandas as pd
from dotenv import load_dotenv

In [4]:
# # Create Ollama language model - Gemma 2
local_llm = 'llama3.2:latest'

llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=512,  
                 temperature=0)

In [10]:
import os

In [11]:
load_dotenv() 
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [12]:
llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY,
                        openai_api_type=OPENAI_API_TYPE,
                        #deployment_name = 'genai-gpt-4-32k',
                        #model_name = 'gpt-4-32k')
                        deployment_name = 'genai-gpt-35-turbo',
                        model_name = 'gpt-35-turbo')

C:\Users\750040697\AppData\Local\Temp\ipykernel_23732\3925094818.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(openai_api_base=OPENAI_API_BASE,


In [13]:
df = pd.read_csv(r'./dummy_CAR.csv')
df.head()

,roster_date,RFP_Key,rfp_type_random,rfp_name_random,rfp_location,Name,Address,city,State,Zip,...,program_date,active_quarters,base_active_quarters,sap,location_enddate,location_startdate,eligibility_enddate,eligibility_startdate,startdate,enddate
0,10/2/2024,SCG208815,myAPP,myAPP,47409,Campbell Ltd,34768 Stephen Square,Littleton,West Virginia,45273,...,9/8/2022,7.0,3,28685,4/11/2262,7/1/2024,4/11/2262,7/1/2024,7/1/2024,12/31/2024
1,10/2/2024,SCG353145,myAPP,myAPP,47409,Ferguson PLC,8886 Buchanan Pass,Littleton,Arkansas,4223,...,9/8/2022,7.0,3,28685,4/11/2262,7/1/2024,4/11/2262,7/1/2024,7/1/2024,12/31/2024
2,10/2/2024,SCG562310,myAPP,myAPP,34098,"Singh, Clark and Kramer",80722 Roger Spur Apt. 719,Greenville,Louisiana,19546,...,NaN,NaN,1,36384,4/11/2262,7/1/2024,4/11/2262,7/1/2024,10/1/2024,12/31/2024
3,10/2/2024,SCG191986,myAPP,myAPP,31994,Ramos-Sullivan,07811 Andrea Inlet Apt. 646,Pittsburgh,Georgia,2331,...,NaN,NaN,3,31994,4/11/2262,7/1/2024,4/11/2262,7/1/2024,10/1/2024,12/31/2024
4,10/2/2024,SCG627730,GPO,ACUALL,44348,"Carson, Kim and Watkins",0152 Walters Valley Suite 312,Loganville,Maine,15598,...,9/15/2020,7.0,3,44348,4/11/2262,7/1/2024,4/11/2262,7/1/2024,7/1/2024,12/31/2024


In [102]:
# os.environ['OPENAI_API_KEY'] = ''

In [24]:
agent1 = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,allow_dangerous_code=True
)

In [26]:
agent1 .invoke("can you find if this SAP is present in the roster 36384, also find out the rfp name associated")



> Entering new AgentExecutor chain...


APIConnectionError: Connection error.

In [88]:
df[(df['SAP'] == 6789065)]

,roster_date,RFP_Key,RFP_Name,RFP_Type,RFP_Group,RFP_Location,Name,Address,City,State,Zip,Phone,RFP_Primary,program_flag,payment_flag,program_date,active_quarters,base_active_quarters,SAP,Location_EndDate,Location_StartDate,Eligibility_EndDate,Eligibility_StartDate,Startdate,Enddate
4,09/24/2024,SCG989459,HAL,ACUALL,58678030,6105994,Taylor-Wallace,1284 Tanner Walks,East Justin,Louisiana,84261,+1-911-666-6594x37647,0,Preferred,1.0,NaN,NaN,1,6789065,04/11/2262,07/01/2024,04/11/2262,07/01/2024,10/01/2024,12/31/2024
